# Pre-procesamiento y selección de características

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression

In [2]:
df1 = pd.read_csv('dataset-part1.csv')

In [3]:
df2 = pd.read_csv('dataset-part2.csv')

In [4]:
frames = [df1, df2]
df_final = pd.concat(frames)

In [5]:
df_final.shape

(12207873, 50)

In [6]:
df_final.head()

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,40,0,1616660040,120,339,0,1616660040010,1616660040,...,1460,0,0,0,0,0,Unknown,1,2293398,Normal flow
1,1,0,",",0,1616660040,120,0,0,1616660040068,1616660040,...,0,0,0,0,40,0,ICMP,1,2293400,Normal flow
2,1,0,104,0,1616660040,120,44725,44,1616660040114,1616660040,...,0,0,0,0,0,0,TLS,1,2293404,Normal flow
3,1,0,",",0,1616660040,120,0,0,1616660040122,1616660040,...,1440,0,8,0,40,0,Unknown,1,2293407,Normal flow
4,1,0,40,0,1616660040,120,1114,1,1616660040184,1616660040,...,0,0,0,0,0,0,TLS,1,2293409,Normal flow


In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12207873 entries, 0 to 8637206
Data columns (total 50 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   BIFLOW_DIRECTION            int64 
 1   DIRECTION                   int64 
 2   DST_TO_SRC_SECOND_BYTES     object
 3   FIREWALL_EVENT              int64 
 4   FIRST_SWITCHED              int64 
 5   FLOW_ACTIVE_TIMEOUT         int64 
 6   FLOW_DURATION_MICROSECONDS  int64 
 7   FLOW_DURATION_MILLISECONDS  int64 
 8   FLOW_END_MILLISECONDS       int64 
 9   FLOW_END_SEC                int64 
 10  FLOW_ID                     int64 
 11  FLOW_INACTIVE_TIMEOUT       int64 
 12  FLOW_START_MILLISECONDS     int64 
 13  FLOW_START_SEC              int64 
 14  FRAME_LENGTH                int64 
 15  IN_BYTES                    int64 
 16  IN_PKTS                     int64 
 17  IPV4_DST_ADDR               object
 18  IPV4_SRC_ADDR               object
 19  L4_DST_PORT                 int64 
 20  L

In [8]:
df_final.describe()

,BIFLOW_DIRECTION,DIRECTION,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,FLOW_ID,...,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP
count,12207873.0,1.220787e+07,12207873.0,1.220787e+07,12207873.0,1.220787e+07,1.220787e+07,1.220787e+07,1.220787e+07,1.220787e+07,...,1.220787e+07,1.220787e+07,1.220787e+07,1.220787e+07,1.220787e+07,1.220787e+07,1.220787e+07,1.220787e+07,12207873.0,1.220787e+07
mean,1.0,2.158525e-03,0.0,1.617770e+09,120.0,2.162737e+07,2.162695e+04,1.617770e+12,1.617770e+09,5.890479e+07,...,1.729981e+04,7.105164e+03,7.082952e+02,2.986968e+02,1.946507e+00,1.632270e+00,9.030550e+00,5.588796e+00,1.0,5.890479e+07
std,0.0,4.640976e-02,0.0,7.090651e+05,0.0,3.997485e+07,3.997481e+04,7.090782e+08,7.090782e+05,3.531791e+07,...,2.771698e+04,1.481242e+04,7.240367e+02,5.977433e+02,3.211679e+00,3.208334e+00,3.387427e+01,2.379983e+01,0.0,3.531791e+07
min,1.0,0.000000e+00,0.0,1.616660e+09,120.0,0.000000e+00,0.000000e+00,1.616660e+12,1.616660e+09,2.293398e+06,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,2.293398e+06
25%,1.0,0.000000e+00,0.0,1.616673e+09,120.0,4.610000e+02,0.000000e+00,1.616673e+12,1.616673e+09,5.345536e+06,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,5.345536e+06
50%,1.0,0.000000e+00,0.0,1.618222e+09,120.0,2.998000e+04,2.900000e+01,1.618222e+12,1.618222e+09,7.978476e+07,...,1.024000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,7.978476e+07
75%,1.0,0.000000e+00,0.0,1.618232e+09,120.0,1.373936e+07,1.373900e+04,1.618232e+12,1.618232e+09,8.283673e+07,...,6.424000e+04,5.010000e+02,1.460000e+03,0.000000e+00,7.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,8.283673e+07
max,1.0,1.000000e+00,0.0,1.618262e+09,120.0,1.199999e+08,1.199990e+05,1.618263e+12,1.618263e+09,8.588870e+07,...,6.553500e+04,6.553500e+04,8.974000e+03,6.547500e+04,1.400000e+01,1.400000e+01,2.530000e+02,2.530000e+02,1.0,8.588870e+07


In [9]:
df_final.isnull().sum()

BIFLOW_DIRECTION              0
DIRECTION                     0
DST_TO_SRC_SECOND_BYTES       0
FIREWALL_EVENT                0
FIRST_SWITCHED                0
FLOW_ACTIVE_TIMEOUT           0
FLOW_DURATION_MICROSECONDS    0
FLOW_DURATION_MILLISECONDS    0
FLOW_END_MILLISECONDS         0
FLOW_END_SEC                  0
FLOW_ID                       0
FLOW_INACTIVE_TIMEOUT         0
FLOW_START_MILLISECONDS       0
FLOW_START_SEC                0
FRAME_LENGTH                  0
IN_BYTES                      0
IN_PKTS                       0
IPV4_DST_ADDR                 0
IPV4_SRC_ADDR                 0
L4_DST_PORT                   0
L4_SRC_PORT                   0
LAST_SWITCHED                 0
MAX_IP_PKT_LEN                0
MIN_IP_PKT_LEN                0
OOORDER_IN_PKTS               0
OOORDER_OUT_PKTS              0
OUT_BYTES                     0
OUT_PKTS                      0
PROTOCOL                      0
PROTOCOL_MAP                  0
RETRANSMITTED_IN_BYTES        0
RETRANSM

In [10]:
df_final.dtypes

BIFLOW_DIRECTION               int64
DIRECTION                      int64
DST_TO_SRC_SECOND_BYTES       object
FIREWALL_EVENT                 int64
FIRST_SWITCHED                 int64
FLOW_ACTIVE_TIMEOUT            int64
FLOW_DURATION_MICROSECONDS     int64
FLOW_DURATION_MILLISECONDS     int64
FLOW_END_MILLISECONDS          int64
FLOW_END_SEC                   int64
FLOW_ID                        int64
FLOW_INACTIVE_TIMEOUT          int64
FLOW_START_MILLISECONDS        int64
FLOW_START_SEC                 int64
FRAME_LENGTH                   int64
IN_BYTES                       int64
IN_PKTS                        int64
IPV4_DST_ADDR                 object
IPV4_SRC_ADDR                 object
L4_DST_PORT                    int64
L4_SRC_PORT                    int64
LAST_SWITCHED                  int64
MAX_IP_PKT_LEN                 int64
MIN_IP_PKT_LEN                 int64
OOORDER_IN_PKTS                int64
OOORDER_OUT_PKTS               int64
OUT_BYTES                      int64
O

In [11]:
for i in df_final.columns:
    print(i + ' - ' + str(len(df_final[i].unique())))

BIFLOW_DIRECTION - 1
DIRECTION - 2
DST_TO_SRC_SECOND_BYTES - 569243
FIREWALL_EVENT - 1
FIRST_SWITCHED - 101845
FLOW_ACTIVE_TIMEOUT - 1
FLOW_DURATION_MICROSECONDS - 3285923
FLOW_DURATION_MILLISECONDS - 104699
FLOW_END_MILLISECONDS - 7829766
FLOW_END_SEC - 101898
FLOW_ID - 12207873
FLOW_INACTIVE_TIMEOUT - 1
FLOW_START_MILLISECONDS - 7753548
FLOW_START_SEC - 101845
FRAME_LENGTH - 1
IN_BYTES - 76968
IN_PKTS - 8575
IPV4_DST_ADDR - 440887
IPV4_SRC_ADDR - 88463
L4_DST_PORT - 65536
L4_SRC_PORT - 65118
LAST_SWITCHED - 101898
MAX_IP_PKT_LEN - 1
MIN_IP_PKT_LEN - 1
OOORDER_IN_PKTS - 21
OOORDER_OUT_PKTS - 26
OUT_BYTES - 91938
OUT_PKTS - 9597
PROTOCOL - 5
PROTOCOL_MAP - 5
RETRANSMITTED_IN_BYTES - 2983
RETRANSMITTED_IN_PKTS - 446
RETRANSMITTED_OUT_BYTES - 8643
RETRANSMITTED_OUT_PKTS - 1626
SRC_TO_DST_SECOND_BYTES - 3160289
TCP_FLAGS - 54
TCP_WIN_MAX_IN - 16345
TCP_WIN_MAX_OUT - 11685
TCP_WIN_MIN_IN - 16455
TCP_WIN_MIN_OUT - 10863
TCP_WIN_MSS_IN - 135
TCP_WIN_MSS_OUT - 63
TCP_WIN_SCALE_IN - 15
TCP_WIN

Descartamos columnas con un único valor, ya que si para todos los datos es el mismo valor, no está aportando nueva información al modelo

In [12]:
df_final.drop(['SAMPLING_INTERVAL', 'MIN_IP_PKT_LEN', 'MAX_IP_PKT_LEN', 'FRAME_LENGTH', 'FLOW_INACTIVE_TIMEOUT', 
               'FLOW_ACTIVE_TIMEOUT', 'FIREWALL_EVENT', 'BIFLOW_DIRECTION'
              ], axis=1, inplace=True)

Descartamos columnas que están en microsegundos y en segundos, ya que tenemos sus valores correspondientes en milisegundos, dado que la mayor cantidad de características está en milisegundos, se optó por esta medida 

In [13]:
df_final.drop(['FLOW_DURATION_MICROSECONDS', 'FLOW_END_SEC', 'FLOW_START_SEC'], axis=1, inplace=True)

Descartamos algunas columnas de las cuales por el tipo de información contenida se considera dificil de extraer información valiosa

In [14]:
df_final.drop(['L7_PROTO_NAME', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES', 
               'FLOW_ID', 'IPV4_DST_ADDR', 'IPV4_SRC_ADDR'
              ], axis=1, inplace=True)

In [15]:
var_dummies = ['PROTOCOL_MAP', 'PROTOCOL']

for i in var_dummies:
    # convertir a variables dummies
    value_dummies = pd.get_dummies(df_final[i], prefix=i, drop_first=True)
    df_final = pd.concat([df_final.drop(i, axis=1), value_dummies], axis=1)

df_final.columns

Index(['DIRECTION', 'FIRST_SWITCHED', 'FLOW_DURATION_MILLISECONDS',
       'FLOW_END_MILLISECONDS', 'FLOW_START_MILLISECONDS', 'IN_BYTES',
       'IN_PKTS', 'L4_DST_PORT', 'L4_SRC_PORT', 'LAST_SWITCHED',
       'OOORDER_IN_PKTS', 'OOORDER_OUT_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS', 'TCP_FLAGS',
       'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT', 'TCP_WIN_MIN_IN',
       'TCP_WIN_MIN_OUT', 'TCP_WIN_MSS_IN', 'TCP_WIN_MSS_OUT',
       'TCP_WIN_SCALE_IN', 'TCP_WIN_SCALE_OUT', 'SRC_TOS', 'DST_TOS',
       'TOTAL_FLOWS_EXP', 'LABEL', 'PROTOCOL_MAP_icmp',
       'PROTOCOL_MAP_ipv6-icmp', 'PROTOCOL_MAP_tcp', 'PROTOCOL_MAP_udp',
       'PROTOCOL_6', 'PROTOCOL_17', 'PROTOCOL_47', 'PROTOCOL_58'],
      dtype='object')

In [16]:
x = df_final.drop(['LABEL'], axis=1)
y = df_final['LABEL']

Evaluamos las caracteristicas para ver cuales son las mas apropiadas a utilizar

In [17]:
select = SelectKBest(score_func=chi2, k=15)
z = select.fit_transform(x, y)

In [18]:
filter = select.get_support()
features = list(x.columns)

In [19]:
selected_features = []

for i in range(0, len(features)):
    if filter[i]:
        print(features[i])
        selected_features.append(features[i])

FLOW_DURATION_MILLISECONDS
FLOW_END_MILLISECONDS
FLOW_START_MILLISECONDS
IN_BYTES
L4_DST_PORT
L4_SRC_PORT
OUT_BYTES
RETRANSMITTED_OUT_BYTES
TCP_WIN_MAX_IN
TCP_WIN_MAX_OUT
TCP_WIN_MIN_IN
TCP_WIN_MIN_OUT
TCP_WIN_MSS_IN
TCP_WIN_MSS_OUT
TOTAL_FLOWS_EXP


Convertimos nuestro target a una variable categórica

In [20]:
codes = {'Normal flow': 0, 'SYN Scan - aggressive': 1, 'Denial of Service R-U-Dead-Yet': 2, 'Denial of Service Slowloris': 3}

df_final['LABEL'] = df_final['LABEL'].astype('category')
df_final['LABEL'] = df_final['LABEL'].map(codes)

In [21]:
df_final['LABEL']

0          0
1          0
2          0
3          0
4          0
          ..
8637202    0
8637203    0
8637204    0
8637205    0
8637206    0
Name: LABEL, Length: 12207873, dtype: category
Categories (4, int64): [2, 3, 0, 1]

In [22]:
selected_features.append('LABEL')

In [23]:
print(selected_features)

['FLOW_DURATION_MILLISECONDS', 'FLOW_END_MILLISECONDS', 'FLOW_START_MILLISECONDS', 'IN_BYTES', 'L4_DST_PORT', 'L4_SRC_PORT', 'OUT_BYTES', 'RETRANSMITTED_OUT_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT', 'TCP_WIN_MIN_IN', 'TCP_WIN_MIN_OUT', 'TCP_WIN_MSS_IN', 'TCP_WIN_MSS_OUT', 'TOTAL_FLOWS_EXP', 'LABEL']


In [24]:
df = df_final[selected_features]

In [25]:
df.head()

,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_START_MILLISECONDS,IN_BYTES,L4_DST_PORT,L4_SRC_PORT,OUT_BYTES,RETRANSMITTED_OUT_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TOTAL_FLOWS_EXP,LABEL
0,0,1616660040010,1616660040010,44,60541,22351,40,0,1024,0,1024,0,1460,0,2293398,0
1,0,1616660040068,1616660040068,68,0,0,0,0,0,0,0,0,0,0,2293400,0
2,44,1616660040114,1616660040070,189,24650,443,104,0,122,1634,122,1634,0,0,2293404,0
3,0,1616660040122,1616660040122,52,8728,61075,0,0,8192,0,8192,0,1440,0,2293407,0
4,1,1616660040184,1616660040183,189,443,39762,40,0,502,0,502,0,0,0,2293409,0


Extraemos un dataset unicamente con las variables que nos interesan

In [26]:
df.to_csv('datafinal.csv')